In [24]:
import src.DetectingRegionInfo as DetectingRegionInfo
import src.DopplerInfo as DopplerInfo

import src.LinesGenerator as LinesGenerator
import src.WAndDopplerGenerator as WAndDopplerGenerator
import src.FeaturesAndLabelsGenerator as FeaturesAndLabelsGenerator

import src.Config as Config
from matplotlib.patches import Polygon
import matplotlib.pyplot as plt

# create info structs from config
detecting_region_info = DetectingRegionInfo.DetectingRegionInfo(
    Config.transmittor_position, Config.receiver_position_1, Config.receiver_position_2, Config.receiver_position_3)
doppler_info = DopplerInfo.DopplerInfo(Config.c, Config.fc, Config.v)

In [25]:
def visualize_lines(detecting_region_info, lines):
    _, ax = plt.subplots()
    ax.set_xlim(0, 300)
    ax.set_ylim(0, 150)
    ax.add_patch(Polygon([detecting_region_info.v1,
                 detecting_region_info.v2, detecting_region_info.v4, detecting_region_info.v3], fill=False))

    # Plot the lines
    for i in range(len(lines)):
        line = lines[i]
        ax.plot([point[0] for point in line], [point[1]
                for point in line], "r")

    plt.show()


In [ ]:
# training features and labels

[lines_a, lines_b] = LinesGenerator.generateLines(
    detecting_region_info=detecting_region_info,
    a_b_distance=Config.a_b_distance,
    num_of_lines_to_generate=Config.train_num_of_lines_to_generate,
    step_count_per_line=Config.step_count_per_line,
    length_per_step=Config.length_per_step,
    angle_change_limit_per_step=Config.angle_change_limit_per_step
)

[w, doppler] = WAndDopplerGenerator.generateWAndDoppler(
    detecting_region_info=detecting_region_info,
    doppler_info=doppler_info,
    lines_a=lines_a,
    lines_b=lines_b
)

[train_features, train_labels] = FeaturesAndLabelsGenerator.generateFeaturesAndLabels(
    detecting_region_info=detecting_region_info,
    lines_a=lines_a,
    w=w,
    doppler=doppler,
    num_of_lines_to_generate=Config.train_num_of_lines_to_generate,
    step_count_per_line=Config.step_count_per_line
)

train_features = train_features[:, -1, :]
# torch.Size([100, 5, 6] -> torch.Size([100, 6])

visualize_lines(detecting_region_info, lines_a)

In [ ]:
# short testing features and labels

[lines_a, lines_b] = LinesGenerator.generateLines(
    detecting_region_info=detecting_region_info,
    a_b_distance=Config.a_b_distance,
    num_of_lines_to_generate=Config.test_set_short_num,
    step_count_per_line=Config.step_count_per_line,
    length_per_step=Config.length_per_step,
    angle_change_limit_per_step=Config.angle_change_limit_per_step
)

[w, doppler] = WAndDopplerGenerator.generateWAndDoppler(
    detecting_region_info=detecting_region_info,
    doppler_info=doppler_info,
    lines_a=lines_a,
    lines_b=lines_b
)

[test_short_features, test_short_labels] = FeaturesAndLabelsGenerator.generateFeaturesAndLabels(
    detecting_region_info=detecting_region_info,
    lines_a=lines_a,
    w=w,
    doppler=doppler,
    num_of_lines_to_generate=Config.test_set_short_num,
    step_count_per_line=Config.step_count_per_line
)

test_short_features = test_short_features[:, -1, :]

visualize_lines(detecting_region_info, lines_a)

In [ ]:
import torch

# train_features.shape, train_labels.shape

# make train_features and train_labels into a dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = {}
dataset['train_input'] = torch.tensor(train_features, dtype=torch.float64).to(device)
dataset['test_input'] = torch.tensor(test_short_features, dtype=torch.float64).to(device)

dataset['train_label'] = torch.tensor(train_labels, dtype=torch.float64).to(device)
dataset['test_label'] = torch.tensor(test_short_labels, dtype=torch.float64).to(device)

# (torch.Size([100, 5, 6]), torch.Size([100, 2]))
dataset['train_input'].shape, dataset['train_label'].shape


In [ ]:
from src.kan import *
torch.set_default_dtype(torch.float64)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# create a KAN: 2D inputs, 1D output, and 5 hidden neurons. cubic spline (k=3), 5 grid intervals (grid=5).
model = KAN(width=[6,5,2], grid=3, k=3, seed=42, device=device)

In [ ]:
# plot KAN at initialization
model(dataset['train_input'])
model.plot()

In [ ]:
# train the model
model.fit(dataset, opt="LBFGS", steps=50, lamb=0.001)

In [ ]:
model.plot()

In [ ]:
model = model.prune()
model.plot()

In [ ]:
model.fit(dataset, opt="LBFGS", steps=50)

In [ ]:
model = model.refine(10)

In [ ]:
model.fit(dataset, opt="LBFGS", steps=50)

Automatically or manually set activation functions to be symbolic

In [ ]:
mode = "auto" # "manual"

if mode == "manual":
    # manual mode
    model.fix_symbolic(0,0,0,'sin')
    model.fix_symbolic(0,1,0,'x^2')
    model.fix_symbolic(1,0,0,'exp')
elif mode == "auto":
    # automatic mode
    lib = ['x','x^2','x^3','x^4','exp','log','sqrt','tanh','sin','abs']
    model.auto_symbolic(lib=lib)

In [ ]:
model.fit(dataset, opt="LBFGS", steps=50)

In [ ]:
from src.kan.utils import ex_round

ex_round(model.symbolic_formula()[0][0],4)